http://www.kfish.org/boids/pseudocode.html (no good)

https://www.red3d.com/cwr/papers/1999/gdc99steer.pdf

# Separation
Stee to avoid crowding local flockmates

In [21]:
import logging
logger = logging.getLogger('matplotlib')
logger.setLevel(logging.INFO)

import io
import base64
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML, Image, Video
import numpy as np
#animation.verbose.set_level('debug')

In [2]:
class boids:
    def __init__(self,size,center=[0,0,0],distance=.1,velocity=1e-3):
        self.mass=1
        self.size=size
        l=size**(1/3)*distance
        #positions
        self.x=center[0]+l*np.random.rand(size,1)-l/2
        self.y=center[1]+l*np.random.rand(size,1)-l/2
        self.z=center[2]+l*np.random.rand(size,1)-l/2
        #velocities
        self.u=velocity*np.random.rand(size,1)-velocity/2
        self.v=velocity*np.random.rand(size,1)-velocity/2
        self.w=velocity*np.random.rand(size,1)-velocity/2
        #limiting to xy plane
        self.z=self.z*0
        self.w=self.z
        self.cyclicX=True
        self.cyclicY=True
        self.cyclicZ=False
        self.boxLx=.2
        self.boxLy=.2
        self.boxLz=0
        self.maxForce=0.1
        self.maxSpeed=1
        #orientation is in this example just the unit vector pointing in the direction os the velocity with no roll angle
    
    def setPosition(x,y,z):
        self.x=x
        self.y=y
        self.z=z
        
    def updateDistance(self):
        px=self.x*np.ones((self.size,1))
        self.dx=px.T-px
        if self.cyclicX:
            self.dx+=(self.dx<(-self.boxLx/2))*self.boxLx
            self.dx-=(self.dx>( self.boxLx/2))*self.boxLx
            
        py=self.y*np.ones((self.size,1))
        self.dy=py.T-py
        if self.cyclicY:
            self.dy+=(self.dy<(-self.boxLy/2))*self.boxLy
            self.dy-=(self.dy>( self.boxLy/2))*self.boxLy
        pz=self.z*np.ones((self.size,1))
        self.dz=pz.T-pz
        if self.cyclicZ:
            self.dz+=(self.dz<(-self.boxLz/2))*self.boxLz
            self.dz-=(self.dz>( self.boxLz/2))*self.boxLz
        self.sqDist=np.square(self.dx)+np.square(self.dy)+np.square(self.dz)
        self.dist=np.sqrt(self.sqDist)
        
    def updateOrientation(self):
        l=np.sqrt(np.square(self.u)+np.square(self.v)+np.square(self.w))
        self.fx=self.u/l
        self.fy=self.v/l
        self.fz=self.w/l
        
    def inNeighborhood(self,minDist,maxDist,angle):
        inSight=((self.fx*self.dx+self.fy*self.dy+self.fz*self.dz+np.eye(self.size))/(self.dist+np.eye(self.size)))>np.cos(np.radians(angle))
        return np.logical_and(
            np.logical_or(
                np.logical_and(self.dist<minDist,self.dist>0.0),
                np.logical_and(inSight,(self.dist<maxDist))
            ),
            np.eye(ff.size)==0
        )
        
    def separation(self,minDist,maxDist,angle):
        isN=self.inNeighborhood(minDist,maxDist,angle)
        sepX=-np.sum(isN*self.dx/(self.sqDist+np.eye(self.size)),1,keepdims=True)
        sepY=-np.sum(isN*self.dy/(self.sqDist+np.eye(self.size)),1,keepdims=True)
        sepZ=-np.sum(isN*self.dz/(self.sqDist+np.eye(self.size)),1,keepdims=True)
        return sepX,sepY,sepZ,
        
    def steer(self,xForce,yForce,zForce,timeStep):
        #limit force maxForce
        l=np.sqrt(np.square(xForce)+np.square(yForce)+np.square(zForce))
        scaler=np.clip(self.maxForce/l,0,1.)
        xForce=xForce*scaler
        yForce=yForce*scaler
        zForce=zForce*scaler
#        print(xForce)
        #calculate acceleration
        xAcc=xForce/self.mass
        yAcc=yForce/self.mass
        zAcc=zForce/self.mass
        #update velocity
        u=self.u+xAcc*timeStep
        v=self.v+yAcc*timeStep
        w=self.w+zAcc*timeStep
        #limit velocity to maxSpeed
        l=np.sqrt(np.square(self.u)+np.square(self.v)+np.square(self.w))
        scaler=np.clip(self.maxSpeed/l,0,1.)
#        print(scaler)
#        print(xAcc)
        self.u=u*scaler
        self.v=v*scaler
        self.w=w*scaler
#        print(self.u)
        self.x=self.x+self.u*timeStep
        self.y=self.y+self.v*timeStep
        self.z=self.z+self.w*timeStep
        # apply boundaries
        if self.cyclicX:
            self.x-=(self.x>( self.boxLx/2))*self.boxLx
            self.x+=(self.x<(-self.boxLx/2))*self.boxLx
        if self.cyclicY:
            self.y-=(self.y>( self.boxLy/2))*self.boxLy
            self.y+=(self.y<(-self.boxLy/2))*self.boxLy
        if self.cyclicZ:
            self.z-=(self.z>( self.boxLz/2))*self.boxLz
            self.z+=(self.z<(-self.boxLz/2))*self.boxLz
        self.updateDistance()
        self.updateOrientation()
        
        
        


In [3]:
rc('animation', html='html5')

In [ ]:
ff=boids(3)
ff.x=np.array([[-.09,0,.09]]).T
ff.y=np.array([[-0.001,0,0.001]]).T
ff.u=np.array([[1,1,1]]).T
ff.v=np.array([[0,0,0]]).T
ff.updateDistance()
ff.updateOrientation()
print(ff.inNeighborhood(0.01,0.05,25))
sepX,sepY,sepZ=ff.separation(0.01,0.05,25)
print(.1*sepX)
print(.1*sepY)
for i in range(1000):
    print(i)
    ff.steer(np.array([[1,0,0]]).T,np.array([[0,0,0]]).T,np.array([[0,0,0]]).T,0.01)
    print(ff.x)

In [ ]:
print(ff.dist)
print(ff.dx)


In [ ]:
ff.inNeighborhood(0.01,0.05,25)

In [ ]:
print(ff.inNeighborhood(0.01,0.05,25)*ff.dx)
print(ff.inNeighborhood(0.01,0.05,25)*ff.dy)
print(-np.sum(ff.inNeighborhood(0.01,0.05,25)*ff.dx,1,keepdims=True))
print(-np.sum(ff.inNeighborhood(0.01,0.05,25)*ff.dy,1,keepdims=True))
ff.sqDist

In [ ]:
ff.dist

In [ ]:
%matplotlib notebook
ax = plt.axes()
#fg = plt.figure()
#plt.plot(ff.x,ff.y,"x")
plt.xlim(-.1,.1)
plt.ylim(-.1,.1)
#plt.grid()
plt.axes().set_aspect('equal', 'datalim')
r=0.05
rmin=0.01
vang=25
patches=[]
colors=[]
for i in range(ff.size):
    ang=np.arctan(ff.v[i]/ff.u[i])
    if ff.u[i]<0:
        ang+=np.pi
    ang*=(180/np.pi)
    print((ff.x[i], ff.y[i]), r,ang-vang,ang+vang)
    patches += [mpatches.Wedge((ff.x[i][0], ff.y[i][0]), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0], ff.y[i][0]), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0], ff.y[i][0]-.2), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0], ff.y[i][0]-.2), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0], ff.y[i][0]+.2), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0], ff.y[i][0]+.2), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0]-.2, ff.y[i][0]), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0]-.2, ff.y[i][0]), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0]-.2, ff.y[i][0]-.2), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0]-.2, ff.y[i][0]-.2), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0]-.2, ff.y[i][0]+.2), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0]-.2, ff.y[i][0]+.2), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0]+.2, ff.y[i][0]), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0]+.2, ff.y[i][0]), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0]+.2, ff.y[i][0]-.2), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0]+.2, ff.y[i][0]-.2), rmin)]
    patches += [mpatches.Wedge((ff.x[i][0]+.2, ff.y[i][0]+.2), r,ang-vang,ang+vang)]
    patches += [mpatches.Circle((ff.x[i][0]+.2, ff.y[i][0]+.2), rmin)]
    tt=ff.inNeighborhood(rmin,r,vang)
    if any(tt[i]):
        colors += [.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9,.9]
    else:
        colors += [0.5,0.5,.5,.5,0.5,0.5,.5,.5,.5,0.5,0.5,.5,.5,0.5,0.5,.5,.5,.5]
p = PatchCollection(patches, cmap=matplotlib.cm.jet, alpha=0.2)
p.set_array(np.array(colors))
p.set_clim([0, 1])
ax.add_collection(p)
plt.quiver(ff.x,ff.y,ff.fx,ff.fy,angles='xy', scale_units='xy' )
plt.quiver(ff.x,ff.y,sepX,sepY,color='r',angles='xy', scale_units='xy' )
plt.colorbar(p)
plt.xlim(-.1,.1)
plt.ylim(-.1,.1)


In [ ]:
np.arctan(-1)

In [ ]:
A=np.zeros((20,2,2))
A[0,0,0]=1
A[0,1,1]=1
A[1,0,0]=-1
A[1,1,1]=-1
B=np.zeros((20,2,1))
B[0,0,0]=1
B[1,0,0]=-1
A
B
C=np.matmul(A,B)
print(A.shape)
print(A.T.shape)
print(C.shape)
print(C[0,0],C[1,0],C[0,1],C[1,1])

In [ ]:
n = 4
x = [(i,j) for i in range(1,n+1) for j in range(i+1, n+1)]
print(x)

In [ ]:
import itertools
a=['a','b','c','d','e']
b=itertools.combinations(a,2)
print(list(b))

In [ ]:

from scipy import signal
import matplotlib
%matplotlib notebook
#np.random.seed(101)
#g = np.floor(np.random.random((100, 100)) + .5)

#plt.subplot(211)
#plt.imshow(g)
#plt.subplot(212)
#plt.imshow(g, cmap='Greys',  interpolation='nearest')
#plt.show()

import matplotlib.animation as animation
N=400

#fig = plt.figure()
fig, (mod,cor) = plt.subplots(1,2)
#g = 2*np.floor(np.random.random((N, N))+.5)-1
im=mod.imshow(g, cmap='Greys',  interpolation='nearest')
c= g*0
ci=cor.imshow(c, cmap='Greys',  interpolation='nearest')
norm=matplotlib.colors.Normalize(-N*N,N*N)
ci.set_norm(norm)

im.set_array(g)
S=-1
beta=1e-1
h=0
Ea=1

docor=False

def init():  # only required for blitting to give a clean slate.

    return line,
cyclic=True

def animate(i):
    global g,beta,h,c,docor
    #g = np.floor(np.random.random((100, 100))+.5)
    #beta*=1.002
    for k in range(10000):
        x=np.random.randint(N)
        y=np.random.randint(N)
        gt=-g[x,y]
        dH=2*h*gt  # h*(gt-g[x,y])
        if x!=0:
            dH+=2*S*gt*g[x-1,y] # = S*(gt*g[x-1,y]-g[x,y]*g[x-1,y])
        elif cyclic:
            dH+=2*S*gt*g[N-1,y]
        if x!=(N-1):
            dH+=2*S*gt*g[x+1,y]
        elif cyclic:
            dH+=2*S*gt*g[0,y]
        if y!=0:
            dH+=2*S*gt*g[x,y-1]
        elif cyclic:
            dH+=2*S*gt*g[x,N-1]
        if y!=(N-1):
            dH+=2*S*gt*g[x,y+1]
        elif cyclic:
            dH+=2*S*gt*g[x,0]
        p=np.random.random(1)[0]
        if dH<0:
            dH=0
        if (p)<np.exp(-beta*(dH+Ea)):
            g[x,y]=gt
    im.set_array(g)
    if docor:
        c= signal.correlate2d(g,g,mode='same')
        docor=False
    ci.set_array(c)
    #line.set_ydata(np.sin(x + i / 100))  # update the data.
    #return line,
    return im,ci,


ani = animation.FuncAnimation(
    fig, animate, interval=10, blit=True, save_count=50)

#for i in range(10):
#    animate(1)

# To save the animation, use e.g.
#
# ani.save("movie.mp4")
#
# or
#
# from matplotlib.animation import FFMpegWriter
# writer = FFMpegWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)
# ani.save("movie.mp4", writer=writer)

plt.show()

In [ ]:
x=np.array([[1,2,3,4]])
y=np.array([[1,1,1,1]])

In [ ]:
p=x*(y.T)
np.abs(p-p.T)

In [ ]:
np.ones((5,1))

In [ ]:
s=4
tt=np.random.randint(0,10,(3*s,3*s))
tt

In [ ]:
ss=tt.reshape((3,s,s*3))
ss

In [ ]:
i1=ss.argmin(0).flatten()

In [ ]:
i2=np.repeat(np.arange(s).T,3*s)
i3=np.repeat(np.arange(3*s).T,s).reshape((3*s,s)).T.flatten()
print(i1,i2,i3)
print(i1.shape,i2.shape,i3.shape)

In [ ]:
(ss.flatten()[np.ravel_multi_index([i1,i2.flatten(),i3.flatten()], (3,s,3*s))]).reshape((s,3*s))

In [ ]:
uu=(ss.flatten()[np.ravel_multi_index([i1,i2,i3], (3,s,3*s))]).reshape((s,3*s))
uu

In [ ]:
uu.reshape((s,3,s))

In [ ]:
i1=np.repeat(np.arange(s).T,s)
i2=uu.reshape((s,3,s)).argmin(1).flatten()
i3=np.repeat(np.arange(s).T,s).reshape(s,s).T.flatten()
print(i1,i2,i3)
print(i1.shape,i2.shape,i3.shape)

In [ ]:
(uu.flatten()[np.ravel_multi_index([i1,i2,i3], (s,3,s))]).reshape((s,s))

In [ ]:
kk=np.array([[1,2,3]])
np.vstack((kk,kk,kk))

In [ ]:
1/np.array([[1,2,3]])

In [ ]:
#------------------------------------------------------------
# set up figure and animation
fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(131, aspect='equal', autoscale_on=False,
                     xlim=(-3.2, 3.2), ylim=(-2.4, 2.4))
ax2 = fig.add_subplot(132, aspect='equal', autoscale_on=False,
                     xlim=(-3.2, 3.2), ylim=(-2.4, 2.4))
ax3 = fig.add_subplot(133,xlim=(0, 1000), ylim=(-2.4, 2.4))

# particles holds the locations of the particles
particles, = ax.plot([], [], 'bo', ms=6)
particles2, = ax2.plot([], [], 'bo', ms=6)
avg_line, = ax3.plot([],[],'b')
avg_line2, = ax3.plot([],[],'g')
avg_text  = ax3.text(200,2,str(0))
avg_text2  = ax3.text(200,1,str(0))
avg=[]
avg2=[]
tt=[]

# rect is the box edge
rect = plt.Rectangle(box.bounds[::2],
                     box.bounds[1] - box.bounds[0],
                     box.bounds[3] - box.bounds[2],
                     ec='none', lw=2, fc='none')
ax.add_patch(rect)

rect2 = plt.Rectangle(box2.bounds[::2],
                     box2.bounds[1] - box2.bounds[0],
                     box2.bounds[3] - box2.bounds[2],
                     ec='none', lw=2, fc='none')
ax2.add_patch(rect2)


def init():
    """initialize animation"""
    global box, rect,avg_line,avg_text,rect2,avg_text,avg_text2
    particles.set_data([], [])
    rect.set_edgecolor('none')
    rect2.set_edgecolor('none')
    avg_line.set_data([],[])
    avg_line2.set_data([],[])
    return particles, rect ,particles2,rect2,avg_line,avg_line2,avg_text,avg_text2

def animate(i):
    """perform animation step"""
    global box, rect, dt, ax, fig,avg_line,avg_text, rect2,avg_text,avg_text2
    for _ in range(10):
        box.step(dt)
        box2.step(dt)
        avg.append(np.mean(box.state[:, 1]))
        avg2.append(np.mean(box2.state[:, 1]))
        if len(tt)==0:
            tt.append(dt)
        else:
            tt.append(tt[-1]+dt)

    ms = int(fig.dpi * 2 * box.size * fig.get_figwidth()
             / np.diff(ax.get_xbound())[0])
    ms=6
    # update pieces of the animation
    rect.set_edgecolor('k')
    rect2.set_edgecolor('k')
    particles.set_data(box.state[:, 0], box.state[:, 1])
    particles2.set_data(box2.state[:, 0], box2.state[:, 1])
    
    avgv=np.mean(avg)
    avgv2=np.mean(avg2)
    avg_line.set_data(tt,avg)
    avg_line2.set_data(tt,avg2)
    avg_text.set_text(str(avgv))
    avg_text2.set_text(str(avgv2))
    particles.set_markersize(ms)
    particles2.set_markersize(ms)
    return particles, rect ,particles2,rect2,avg_line,avg_line2,avg_text,avg_text2

ani = animation.FuncAnimation(fig, animate, frames=600,
                              interval=1, blit=True, init_func=init)


# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
#ani.save('particle_box.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

plt.show()




In [6]:
%matplotlib agg
fig, ax = plt.subplots()

ff=boids(3)
ff.updateDistance()
ff.updateOrientation()
ff.cyclicX=False
ff.cyclicY=False

if ff.cyclicX:
    jrange=range(-1,2)
else:
    jrange=[0]

if ff.cyclicY:
    krange=range(-1,2)
else:
    krange=[0]
    
    
ff.x=np.array([[-0.025,-.125,0.05]]).T
ff.y=np.array([[-.125,0,.22]]).T
ff.u=np.array([[0,.01,0]]).T
ff.v=np.array([[.01,0,-.01]]).T
r=0.025
rmin=0.005
vang=110
sepX,sepY,sepZ=ff.separation(rmin,r,vang)
wedges=[]
circles=[]
colors=[]
for i in range(ff.size):
    ang=np.arctan(ff.v[i]/(ff.u[i]+1e-9))
    if ff.u[i]<0:
        ang+=np.pi
    ang*=(180/np.pi)
    ws={}
    cs={}
    for j in jrange:
        w={}
        c={}
        for k in krange:
            pw=mpatches.Wedge((ff.x[i][0]+k*ff.boxLx, ff.y[i][0]+j*ff.boxLy), r,ang-vang,ang+vang)
            pw.set_fc((1,0,0,.1))
            ax.add_patch(pw)
            w[k]=pw
            cw=mpatches.Circle((ff.x[i][0], ff.y[i][0]), rmin)
            cw.set_fc((1,0,0,.1))
            ax.add_patch(cw)
            c[k]=cw
        ws[j]=w
        cs[j]=c
    wedges.append(ws)
    circles.append(cs)
velocities={}
sepForce={}
for j in jrange:
    v={}
    s={}
    for k in krange:
        v[k]=plt.quiver(ff.x+k*ff.boxLx,ff.y+j*ff.boxLy,ff.fx,ff.fy,angles='xy', scale_units='xy', scale=1 )
        s[k]=plt.quiver(ff.x+k*ff.boxLx,ff.y+j*ff.boxLy,sepX,sepY,color='r',angles='xy', scale_units='xy' ,scale=1)
    velocities[j]=v
    sepForce[j]=s
plt.xlim(-.1,.1)
plt.ylim(-.1,.1)

qs=[]

objects=[]
objects+=[velocities[j][k] for k in velocities[j] for j in velocities]
objects+=[sepForce[j][k] for k in sepForce[j] for j in sepForce]
for i in range(ff.size):
    objects+=[circles[i][j][k] for k in circles[i][j] for j in circles[i]]
    objects+=[wedges[i][j][k] for k in wedges[i][j] for j in wedges[i]]

def animate(fi):
    sepX,sepY,sepZ=ff.separation(rmin,r,vang)
    ff.steer(0*np.array([[1e-3,0,1e-4]]).T+0.0001*sepX,
             0*np.array([[0,1e-3,1e-4]]).T+0.0001*sepY,
             np.array([[0,0,0]]).T,0.01)
    testg=0
    for i in range(ff.size):
        ang=np.arctan(ff.v[i]/(ff.u[i]+1e-9))
        if ff.u[i]<0:
            ang+=np.pi
        ang*=(180/np.pi)
        for j in jrange:
            for k in krange:
                circles[i][j][k].center=(ff.x[i][0]+testg*fi/1000+k*ff.boxLx,ff.y[i][0]+j*ff.boxLy)
                wedges[i][j][k].set_center((ff.x[i][0]+testg*fi/1000+k*ff.boxLx,ff.y[i][0]+j*ff.boxLy))
                wedges[i][j][k].set_theta1(ang-vang)
                wedges[i][j][k].set_theta2(ang+vang)
    for j in jrange:
        for k in krange:
            q=velocities[j][k]
            q.set_offsets(np.column_stack([ff.x+k*ff.boxLx+testg*fi/1000,ff.y+j*ff.boxLy]))
            q.set_UVC(1*ff.u,1*ff.v)
            Q=sepForce[j][k]
            Q.set_offsets(np.column_stack([ff.x+k*ff.boxLx+testg*fi/1000,ff.y+j*ff.boxLy]))
            Q.set_UVC(.001*sepX,.001*sepY)
    return objects[0:len(objects)]


def init():
    for i in range(ff.size):
        ang=np.arctan(ff.v[i]/(ff.u[i]+1e-9))
        if ff.u[i]<0:
            ang+=np.pi
        ang*=(180/np.pi)
        for j in jrange:
            for k in krange:
                circles[i][j][k].center=(0+k*ff.boxLx,0+j*ff.boxLy)
                wedges[i][j][k].set_center((0+k*ff.boxLx,0+j*ff.boxLy))
#                pw=mpatches.Wedge((ff.x[i][0]+k*ff.boxLx, ff.y[i][0]+j*ff.boxLy), r,ang-vang,ang+vang)
    for j in jrange:
        for k in krange:
            q=velocities[j][k]
            q.set_offsets(np.column_stack([ff.x+k*ff.boxLx,ff.y+j*ff.boxLy]))
            Q=sepForce[j][k]
            Q.set_offsets(np.column_stack([ff.x+k*ff.boxLx,ff.y+j*ff.boxLy]))
    return objects[0:len(objects)]
#    return circles[0][0][0],wedges[0][0][0],
testing=False
if testing:
    init()
    animate(1)
else:
    ani = animation.FuncAnimation(fig, animate, frames=range(4200), init_func=init,
                              interval=5, blit=False)
    #ani.save('animation.gif', writer='imagemagick', fps=60)
    #ani.save('boids_separation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    #plt.show()

#ani.save('/tmp/movie.mp4', writer='ffmpeg', codec='rawvideo')
ani.save('movie.mp4', writer='ffmpeg', extra_args=['-loglevel', 'verbose'])

/usr/lib/python3.4/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in true_divide


In [22]:
video = io.open('movie.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
<video controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
</video>'''.format(encoded.decode('ascii')))

In [10]:
%ls

animation.gif  boids_separation.mp4  movie.mp4  Untitled.ipynb


In [5]:
ani.save('/tmp/movie.mp4', writer='ffmpeg', extra_args=['-loglevel', 'verbose'])
#ani.save('animation.gif', writer='imagemagick', fps=60)

/usr/lib/python3.4/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in true_divide


ValueError: Invalid file object: <_io.BufferedReader name=48>

In [ ]:
Image(url='animation.gif')

In [ ]:
#pause
ani.event_source.stop()

In [ ]:
[x for x in range(-1,2)]

In [ ]:
u=ff.u
v=ff.v


In [ ]:
print("u:",ff.u-u,ff.u)
print("v:",ff.v-v,ff.v)
u=ff.u
v=ff.v


In [ ]:
range(0,0)

In [ ]:
[velocities[j][k] for k in velocities[j] for j in velocities]+[sepForce[j][k] for k in sepForce[j] for j in sepForce]

In [ ]:
[(wedges[i][j][k] for k in wedges[i][j] for j in wedges[i]) for i in range(3)]

In [ ]:
[circles[0][j][k] for k in circles[0][j] for j in circles[0]]

In [ ]:
circles[0][0][0]

In [20]:
Video('./movie.mp4')